In [9]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import random as rnd

In [2]:
def draw_plot(funcs, labels, interval, points=[]):
    x = np.linspace(interval[0], interval[1], 1000)

    Y = [f(x) for f in funcs]

    plt.figure(figsize=(12, 10))
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('График функции')
    plt.axhline(0, color='black', linewidth=0.8)
    plt.axvline(0, color='black', linewidth=0.8)

    for idx, y in enumerate(Y):
        plt.plot(x, y, label=labels[idx])

    for f in funcs:
        points_y = [f(p) for p in points]
        plt.scatter(points, points_y, c='red', s=100, marker='x')

    delta = (interval[1] - interval[0]) * 0.02
    plt.legend()
    plt.xlim([interval[0] - delta, interval[1] + delta])
    plt.grid(True)
    plt.show()

In [7]:
def map_args(Xs, xk):
    return dict(zip(Xs, xk)) # xs1 -> x0, где x0 это число

### Уравнения

In [3]:
Xs = [
    sp.Symbol('x'),
    sp.Symbol('y')
]

# F1(x, y) = 0
# F2(x, y) = 0
Fs = [
    Xs[1] ** 2 - Xs[0] ** 3 + 1, # y^2 - x^3 +1
    Xs[1] - Xs[0] * Xs[1] ** 3 + 4
]

equations = [
    f'{Fs[0]} = 0',
    f'{Fs[1]} = 0',
]

print(equations)

['-x**3 + y**2 + 1 = 0', '-x*y**3 + y + 4 = 0']


In [13]:
fs_solutions = []

for fs in Fs:
    solution = list(sp.solve(fs, Xs[1]))

    print(f'solution {solution}')

    fs_solutions.append(solution)

print(f'\n{fs_solutions}')

solution [-sqrt(x**3 - 1), sqrt(x**3 - 1)]
solution [-(-1/2 - sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)/3 - 1/(x*(-1/2 - sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)), -(-1/2 + sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)/3 - 1/(x*(-1/2 + sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)), -(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)/3 - 1/(x*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3))]

[[-sqrt(x**3 - 1), sqrt(x**3 - 1)], [-(-1/2 - sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)/3 - 1/(x*(-1/2 - sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)), -(-1/2 + sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)/3 - 1/(x*(-1/2 + sqrt(3)*I/2)*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)), -(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3)/3 - 1/(x*(sqrt(11664/x**2 - 108/x**3)/2 - 54/x)**(1/3))]]


In [14]:
for i, sol_fs in enumerate(fs_solutions):
    sol_f = list(map(lambda x: sp.lambdify(Xs[1:], x), sol_fs))
    sol_l = list(map(lambda x: f'{x}', sol_fs))
    draw_plot(sol_f, sol_l, [0, 2])

TypeError: loop of ufunc does not support argument 0 of type Add which has no callable sqrt method

### Записываем якобиан

In [129]:
Jaks = []

for fs in Fs:
    dfs = []
    for xs in Xs:
        dfs.append(fs.diff(xs, 1))

    Jaks.append(dfs)


In [130]:
print(Jaks)

[[-3*x**2, 2*y], [-y**3, -3*x*y**2 + 1]]


Функция для расчета якобиана в xk

In [112]:
def eval_func(F_matrix, xk):
    vals = []
    arg = map_args(Xs, xk)
    for F_row in F_matrix:
        v = []
        for f in F_row:
            v.append(f.subs(arg))

        vals.append(v)

    result = np.array(vals).astype(float)
    return result if len(F_matrix) > 1 else result[0]

In [113]:
J = eval_func(Jaks, [1, 2])
print(J)

[[ -3.   4.]
 [ -8. -11.]]


In [114]:
v = eval_func([Fs], [1, 2])
print(v)

[ 4. -2.]


In [115]:
print(J.shape)
print(v.shape)

(2, 2)
(2,)


In [116]:
z = np.linalg.solve(J, v)
print(z)

[-0.55384615  0.58461538]


In [131]:
def newton(Fs, xk0, iters=1000, e=1e-3):
    xkn = xk0
    for k in range(iters):
        J = eval_func(Jaks, xkn)
        v = eval_func([Fs], xkn)

        z = np.linalg.solve(J, -v)
        xkn1 = xkn + z

        if np.linalg.norm(xkn1 - xkn) <= e or np.linalg.norm(z) <= e or np.linalg.norm(eval_func([Fs], xkn)) <= e:
            return xkn1, k

        xkn = xkn1

    return xkn, iters

In [132]:
xk0 = [1, 1]

root = newton(Fs, xk0)

In [133]:
print(root)

(array([1.50203905, 1.5455686 ]), 5)
